In [2]:
!pip install --upgrade --force-reinstall --no-cache-dir "trl>=0.14,<0.17" unsloth unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 240.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 220.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 269.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 257.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of unsloth to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 282.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 284.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 247.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 258.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 286.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of unsloth-zoo to determine which version is compatible with other requirements. This

In [1]:
import unsloth
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model_base, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [2]:
from datasets import load_dataset

# Load full dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Split into train/validation
split_dataset = full_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
validation_dataset = split_dataset["test"]

# Shuffle (you can set a seed for reproducibility)
train_dataset = train_dataset.shuffle(seed=42)
validation_dataset = validation_dataset.shuffle(seed=42)

print(train_dataset.num_rows, validation_dataset.num_rows)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

900000 100000


In [3]:
# The instructional prompt template for training
training_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
{}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        # Format the prompt and add the EOS token
        text = training_prompt.format(question, str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)



inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

def formatting_prompts_func_eval(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs   = examples["is_correct"]
    texts = []
    for question, solution in zip(questions, solutions):
        texts.append(inference_prompt.format(question, str(solution)))
    return {"text": texts, "gt": [bool(o) for o in outputs]}


formatted_eval_dataset_full = validation_dataset.map(formatting_prompts_func_eval, batched=True)

# fast validation during training dataset
FAST_EVAL_SIZE = min(1000, len(validation_dataset))
fast_indices = list(range(FAST_EVAL_SIZE))
formatted_eval_dataset_fast = formatted_eval_dataset_full.select(fast_indices)

print("Train / Val sizes:", len(train_dataset), len(validation_dataset))
print("Fast eval size:", FAST_EVAL_SIZE)


Map:   0%|          | 0/900000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Train / Val sizes: 900000 100000
Fast eval size: 1000


In [4]:
model = FastLanguageModel.get_peft_model(
    model_base,
    r = 64, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 128, # A common practice is to set alpha = 2 * r
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth 2025.10.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from transformers import TrainingArguments
import os, torch
from trl import SFTTrainer
from google.colab import drive
from sklearn.metrics import accuracy_score

drive.mount('/content/drive', force_remount=True)
OUTPUT_DIR = "/content/drive/MyDrive/DeepLearning/midterm_project/checkpoints"
os.makedirs(OUTPUT_DIR, exist_ok=True)


n_min = 2

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=32,
    max_steps=3000,
    learning_rate=5e-4,
    warmup_steps=5,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=100,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    seed=42,
    remove_unused_columns=False,
)



trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_train_dataset,
    eval_dataset=formatted_eval_dataset_fast,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=args,
    predict_with_generate=False,
    packing=True
)



Mounted at /content/drive


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/900000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainerCallback
import os, re, shutil

class KeepTopNCallback(TrainerCallback):
    """
    Keep only the top-N checkpoints based on a chosen evaluation metric (e.g. eval_loss).
    This version works reliably with Hugging Face Trainer/TRL SFTTrainer.
    - During evaluation, we record the metric value for the current step.
    - During saving, we look at all existing checkpoints on disk and remove those
      that are not among the top-N best according to the recorded metric.
    """

    def __init__(self, n=3, metric_name="eval_loss", greater_is_better=False):
        self.n = int(n)
        self.metric_name = metric_name
        self.greater_is_better = bool(greater_is_better)
        self.metric_by_step = {}  # Map step → metric value

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        # Record the evaluation metric after each evaluation
        if metrics and self.metric_name in metrics:
            self.metric_by_step[int(state.global_step)] = float(metrics[self.metric_name])

    def on_save(self, args, state, control, **kwargs):
        ckpt_root = args.output_dir
        if not os.path.isdir(ckpt_root):
            return

        # Find all checkpoint directories (e.g., checkpoint-50, checkpoint-100, ...)
        ptn = re.compile(r"^checkpoint-(\d+)$")
        existing = []
        for name in os.listdir(ckpt_root):
            m = ptn.match(name)
            if m:
                step = int(m.group(1))
                path = os.path.join(ckpt_root, name)
                if os.path.isdir(path):
                    existing.append((step, path))
        if not existing:
            return

        # Filter only checkpoints for which we have a recorded metric value
        existing_with_metric = [(s, p, self.metric_by_step.get(s, None)) for s, p in existing]
        existing_with_metric = [(s, p, m) for (s, p, m) in existing_with_metric if m is not None]
        if not existing_with_metric:
            return

        # Sort checkpoints by metric (ascending for loss, descending for accuracy)
        existing_with_metric.sort(key=lambda x: x[2], reverse=self.greater_is_better)
        keep_steps = set(s for s, _, _ in existing_with_metric[: self.n])

        # Always keep the current best model checkpoint (to be safe)
        best_ckpt = getattr(state, "best_model_checkpoint", None)
        if best_ckpt:
            try:
                best_step = int(os.path.basename(best_ckpt).split("-")[-1])
                keep_steps.add(best_step)
            except Exception:
                pass

        # Delete checkpoints not in keep_steps
        for step, path in existing:
            if step not in keep_steps:
                try:
                    shutil.rmtree(path, ignore_errors=True)
                except Exception:
                    pass

    def on_train_end(self, args, state, control, **kwargs):
        # Run one last cleanup at the end of training
        self.on_save(args, state, control)

# ===== Usage =====
# Make sure NOT to set save_total_limit, so the callback has full control.
cb = KeepTopNCallback(
    n=n_min,
    metric_name=(args.metric_for_best_model or "eval_loss"),
    greater_is_better=bool(getattr(args, "greater_is_better", False)),
)
trainer.add_callback(cb)


In [7]:
# resume from last checkpoint
last_ckpt = None
if os.path.isdir(OUTPUT_DIR):
    ckpts = [d for d in os.listdir(OUTPUT_DIR) if d.startswith("checkpoint-")]
    if ckpts:
        ckpts.sort(key=lambda x: int(x.split("-")[-1]))
        last_ckpt = os.path.join(OUTPUT_DIR, ckpts[-1])

trainer.train(resume_from_checkpoint=last_ckpt)

print(trainer.state.log_history)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 900,000 | Num Epochs = 1 | Total steps = 3,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 32
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 32 x 1) = 64
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
2000,0.439700,0.420568
2500,0.418000,0.398618
3000,0.404200,0.388437


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


[{'epoch': 0.0071111111111111115, 'grad_norm': 0.3069359064102173, 'learning_rate': 0.000468561872909699, 'loss': 0.7366, 'step': 100}, {'epoch': 0.014222222222222223, 'grad_norm': 0.2965797185897827, 'learning_rate': 0.0004351170568561873, 'loss': 0.6608, 'step': 200}, {'epoch': 0.021333333333333333, 'grad_norm': 0.2939091920852661, 'learning_rate': 0.0004016722408026756, 'loss': 0.6276, 'step': 300}, {'epoch': 0.028444444444444446, 'grad_norm': 0.28470346331596375, 'learning_rate': 0.0003682274247491639, 'loss': 0.6012, 'step': 400}, {'epoch': 0.035555555555555556, 'grad_norm': 0.31150558590888977, 'learning_rate': 0.0003347826086956522, 'loss': 0.5763, 'step': 500}, {'epoch': 0.035555555555555556, 'eval_loss': 0.5591418743133545, 'eval_runtime': 69.3591, 'eval_samples_per_second': 14.418, 'eval_steps_per_second': 7.209, 'step': 500}, {'epoch': 0.042666666666666665, 'grad_norm': 0.3137113153934479, 'learning_rate': 0.00030133779264214045, 'loss': 0.5515, 'step': 600}, {'epoch': 0.049

In [8]:
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import random

eval_ds = formatted_eval_dataset_full  # 或 formatted_eval_dataset_full

N = min(5000, len(eval_ds))
indices = random.sample(range(len(eval_ds)), N)
subset = eval_ds.select(indices)

print("Eval subset size:", len(subset))


FastLanguageModel.for_inference(model)
model.eval()
device = model.device

def parse_output(response_text):
    output_part = response_text.split("Output:\n")[-1]
    return 'true' in output_part.lower()

predictions, ground_truths = [], []

print(f"Running evaluation on {len(subset)} validation examples...")


with torch.inference_mode(), torch.amp.autocast("cuda", dtype=torch.bfloat16):
    for ex in tqdm(subset):
        prompt_text = ex["text"]
        gt = bool(ex["gt"])

        inputs = tokenizer([prompt_text], return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
        response_text = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

        pred = parse_output(response_text)
        predictions.append(pred)
        ground_truths.append(gt)


accuracy = accuracy_score(ground_truths, predictions)
print("\n################################################")
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print("################################################")


print("\n--- First 5 Predictions ---")
for i in range(min(5, len(predictions))):
    print(f"Example {i}: Prediction = {predictions[i]}, Correct Answer = {ground_truths[i]}")


Eval subset size: 5000
Running evaluation on 5000 validation examples...


100%|██████████| 5000/5000 [18:28<00:00,  4.51it/s]


################################################
Validation Accuracy: 87.28%
################################################

--- First 5 Predictions ---
Example 0: Prediction = True, Correct Answer = True
Example 1: Prediction = False, Correct Answer = False
Example 2: Prediction = True, Correct Answer = True
Example 3: Prediction = True, Correct Answer = True
Example 4: Prediction = False, Correct Answer = False


In [9]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)
submission.to_csv('/content/drive/MyDrive/DeepLearning/midterm_project/checkpoints/submission.csv', index=False)


print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

100%|██████████| 10000/10000 [36:48<00:00,  4.53it/s]



Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.
